# Implementing a Two-Player System in a JavaScript Game

This notebook explores how to implement a two-player system in a JavaScript game using object-oriented programming principles. We'll focus specifically on the implementation in the GameLevelSquares level, examining how player controls, rendering, and collision detection work for multiple players.

## Game Architecture Overview

The game we're analyzing is built with a component-based architecture where:
- `GameObject.js` serves as the base class for all game objects
- `GameLevel.js` manages the creation and updating of game levels
- `GameControl.js` handles game flow and level transitions
- Player classes handle user input and movement

Let's start by examining the GameLevelSquares class, which is responsible for setting up our two-player level.

## 1. Setting Up a Two-Player Level

The `GameLevelSquares` class initializes both players with their own properties and positions. Let's examine how this is structured:

In [ ]:
%%JS

# GameLevelSquares.js (JavaScript code)

import Background from './Background.js';
import PlayerOne from './PlayerOne.js';
import PlayerTwo from './PlayerTwo.js';

class GameLevelSquares {
  constructor(gameEnv) {
    console.log('GameLevelSquares initialized');
    
    // Values dependent on gameEnv.create()
    let width = gameEnv.innerWidth;
    let height = gameEnv.innerHeight;
    
    // Background data
    const background_data = {
        name: 'squares-background',
        greeting: "Welcome to Squares Level!",
        // No src means it will use a default color fill
    };
    
    // Player One data
    const player_one_data = {
        id: 'PlayerOne',
        greeting: "I am Player One!",
        SCALE_FACTOR: 10,
        STEP_FACTOR: 100,
        ANIMATION_RATE: 50,
        INIT_POSITION: { x: width / 4, y: height / 2 },
        velocity: { x: 0, y: 0 }, // Initialize velocity
        pixels: { height: 50, width: 50 },
        hitbox: { widthPercentage: 0.1, heightPercentage: 0.1 },
        keypress: { up: 87, left: 65, down: 83, right: 68 } // W, A, S, D
    };
    
    // Player Two data
    const player_two_data = {
        id: 'PlayerTwo',
        greeting: "I am Player Two!",
        SCALE_FACTOR: 10,
        STEP_FACTOR: 100,
        ANIMATION_RATE: 50,
        INIT_POSITION: { x: 3 * width / 4, y: height / 2 },
        velocity: { x: 0, y: 0 }, // Initialize velocity
        pixels: { height: 50, width: 50 },
        hitbox: { widthPercentage: 0.1, heightPercentage: 0.1 },
        keypress: { up: 73, left: 74, down: 75, right: 76 } // I, J, K, L
    };

    this.classes = [      
      { class: Background, data: background_data },
      { class: PlayerOne, data: player_one_data },
      { class: PlayerTwo, data: player_two_data }
    ];
  }
}

export default GameLevelSquares;

### Key Points about GameLevelSquares Setup

1. **Player Initialization**: The level creates data objects for both PlayerOne and PlayerTwo with different configurations.
2. **Starting Positions**: Players start at different positions on the screen:
   - Player One starts at `(width/4, height/2)` (left side)
   - Player Two starts at `(3*width/4, height/2)` (right side)
3. **Different Control Schemes**: Each player has a different set of keys mapped:
   - Player One uses WASD keys (`87, 65, 83, 68`)
   - Player Two uses IJKL keys (`73, 74, 75, 76`)
4. **Game Objects Array**: Both players, along with the background, are added to the `classes` array which will be instantiated by the game engine.

Now let's look at how the player classes are implemented to handle different control schemes.

## 2. Player Class Implementation

Let's examine the PlayerOne class to understand how player controls are implemented:

In [ ]:
%%JS


# PlayerOne.js (JavaScript code)

import GameObject from './GameObject.js';

class PlayerOne extends GameObject {
    constructor(data = null, gameEnv = null) {
        super(gameEnv);
        this.data = data || {};
        
        // Initialize spriteData for collision detection
        this.spriteData = {
            id: this.data.id || "PlayerOne",
            greeting: this.data.greeting || "I am Player One!",
            reaction: this.data.reaction || (() => console.log("Player One collision"))
        };
        
        // Initialize properties with defaults if not provided in data
        this.keypress = this.data.keypress || {up: 87, left: 65, down: 83, right: 68};
        this.velocity = this.data.velocity || { x: 0, y: 0 };
        this.xVelocity = 5; // Default step size
        this.yVelocity = 5; // Default step size
        this.direction = 'down'; // Default direction
        this.pressedKeys = {}; // active keys array
        
        // Create canvas for the player
        this.canvas = document.createElement("canvas");
        this.canvas.id = this.data.id || "playerOne";
        this.canvas.width = this.data.pixels?.width || 50;
        this.canvas.height = this.data.pixels?.height || 50;
        this.ctx = this.canvas.getContext('2d');
        
        // Add canvas to game container
        document.getElementById("gameContainer").appendChild(this.canvas);
        
        // Set initial position
        this.position = this.data.INIT_POSITION || { x: 0, y: 0 };
        
        // Set size
        this.width = 50;
        this.height = 50;
        
        // Bind event listeners
        this.bindMovementKeyListners();
    }

    /**
     * Binds key event listeners to handle object movement.
     */
    bindMovementKeyListners() {
        addEventListener('keydown', this.handleKeyDown.bind(this));
        addEventListener('keyup', this.handleKeyUp.bind(this));
    }

    /**
     * Handles key down events to change the player's velocity.
     */
    handleKeyDown({ keyCode }) {
        switch (keyCode) {
            case 87: // 'W' key
                this.velocity.y = -this.yVelocity;
                this.direction = 'up';
                break;
            case 65: // 'A' key
                this.velocity.x = -this.xVelocity;
                this.direction = 'left';
                break;
            case 83: // 'S' key
                this.velocity.y = this.yVelocity;
                this.direction = 'down';
                break;
            case 68: // 'D' key
                this.velocity.x = this.xVelocity;
                this.direction = 'right';
                break;
        }
    }

    /**
     * Handles key up events to stop the player's velocity.
     */
    handleKeyUp({ keyCode }) {
        switch (keyCode) {
            case 87: // 'W' key
                if (this.velocity.y < 0) this.velocity.y = 0;
                break;
            case 65: // 'A' key
                if (this.velocity.x < 0) this.velocity.x = 0;
                break;
            case 83: // 'S' key
                if (this.velocity.y > 0) this.velocity.y = 0;
                break;
            case 68: // 'D' key
                if (this.velocity.x > 0) this.velocity.x = 0;
                break;
        }
    }
    
    // Other methods omitted for brevity...
}

export default PlayerOne;

Now let's look at PlayerTwo, which has a similar structure but different controls:

In [ ]:
%%JS


# PlayerTwo.js (JavaScript code - focus on the key handling)

/**
 * Handles key down events to change the player's velocity.
 */
handleKeyDown({ keyCode }) {
    switch (keyCode) {
        case 73: // 'I' key (up)
            this.velocity.y = -this.yVelocity;
            this.direction = 'up';
            break;
        case 74: // 'J' key (left)
            this.velocity.x = -this.xVelocity;
            this.direction = 'left';
            break;
        case 75: // 'K' key (down)
            this.velocity.y = this.yVelocity;
            this.direction = 'down';
            break;
        case 76: // 'L' key (right)
            this.velocity.x = this.xVelocity;
            this.direction = 'right';
            break;
    }
}

/**
 * Handles key up events to stop the player's velocity.
 */
handleKeyUp({ keyCode }) {
    switch (keyCode) {
        case 73: // 'I' key (up)
            if (this.velocity.y < 0) this.velocity.y = 0;
            break;
        case 74: // 'J' key (left)
            if (this.velocity.x < 0) this.velocity.x = 0;
            break;
        case 75: // 'K' key (down)
            if (this.velocity.y > 0) this.velocity.y = 0;
            break;
        case 76: // 'L' key (right)
            if (this.velocity.x > 0) this.velocity.x = 0;
            break;
    }
}

### Key Points about Player Implementation

1. **Separation of Controls**: Each player class implements its own controls using different key codes:
   - PlayerOne responds to WASD (87, 65, 83, 68)
   - PlayerTwo responds to IJKL (73, 74, 75, 76)

2. **Shared Event Handling**: Despite having different control keys, both players:
   - Listen to the same global keyboard events
   - Only respond to their specific key codes
   - Update their own velocity and direction properties

3. **Individual Rendering**: Each player:
   - Creates its own canvas element
   - Has its own drawing context
   - Is positioned independently on the screen

Let's now examine how the player movement and collision handling work.

## 3. Player Movement and Drawing

Each player implements its own `update()`, `draw()`, and `move()` methods. Here's the implementation for PlayerOne (PlayerTwo is almost identical):

In [ ]:
%%JS


# PlayerOne.js movement and drawing methods

/**
 * Updates the player's state.
 */
update() {
    this.move();
    this.draw();
    this.collisionChecks();
}

/**
 * Draws the player on the canvas.
 */
draw() {
    if (!this.ctx) return;
    
    // Clear the canvas
    this.ctx.clearRect(0, 0, this.canvas.width, this.canvas.height);
    
    // Set up styles for player
    this.ctx.fillStyle = 'blue'; // Player One is blue
    this.ctx.fillRect(0, 0, this.canvas.width, this.canvas.height);
    
    // Position the canvas
    this.canvas.style.width = `${this.width}px`;
    this.canvas.style.height = `${this.height}px`;
    this.canvas.style.position = 'absolute';
    this.canvas.style.left = `${this.position.x}px`;
    this.canvas.style.top = `${this.gameEnv.top + this.position.y}px`;
}

/**
 * Moves the player according to velocity and ensures boundaries.
 */
move() {
    // Update position according to velocity
    this.position.x += this.velocity.x;
    this.position.y += this.velocity.y;

    // Ensure the player stays within the canvas boundaries
    if (this.position.y + this.height > this.gameEnv.innerHeight) {
        this.position.y = this.gameEnv.innerHeight - this.height;
        this.velocity.y = 0;
    }
    if (this.position.y < 0) {
        this.position.y = 0;
        this.velocity.y = 0;
    }
    if (this.position.x + this.width > this.gameEnv.innerWidth) {
        this.position.x = this.gameEnv.innerWidth - this.width;
        this.velocity.x = 0;
    }
    if (this.position.x < 0) {
        this.position.x = 0;
        this.velocity.x = 0;
    }
}

### Key Points about Movement and Drawing

1. **Independent Canvas Elements**: Each player is drawn on its own separate canvas element
2. **Different Visual Styles**: 
   - PlayerOne is drawn as a blue square
   - PlayerTwo is drawn as a red square
3. **Position Updates**: 
   - Each player updates its position based on its own velocity
   - Both check for and respect canvas boundaries
4. **Game Loop Integration**: 
   - The `update()` method is called by the game loop
   - It coordinates movement, drawing, and collision detection

## 4. Collision Detection Between Players

A critical part of a two-player game is handling collisions between players. This is managed through the GameObject class, which both player classes inherit from:

In [ ]:
%%JS


# GameObject.js collision handling

/** Collision checks
 * uses Player isCollision to detect hit
 * calls collisionAction on hit
 */
collisionChecks() {
    let collisionDetected = false;

    for (var gameObj of this.gameEnv.gameObjects) {
        if (gameObj.canvas && this != gameObj) {
            this.isCollision(gameObj);
            if (this.collisionData.hit) {
                collisionDetected = true;
                this.handleCollisionEvent();
            }
        }
    }

    if (!collisionDetected) {
        this.state.collisionEvents = [];
    }
}

/** Collision detection method
 * usage: if (object.isCollision(platform)) { // action }
 */
isCollision(other) {
    // Bounding rectangles from Canvas
    const thisRect = this.canvas.getBoundingClientRect();
    const otherRect = other.canvas.getBoundingClientRect();

    // Calculate hitbox constants for this object
    const thisWidthReduction = thisRect.width * (this.hitbox?.widthPercentage || 0.0);
    const thisHeightReduction = thisRect.height * (this.hitbox?.heightPercentage || 0.0);

    // Calculate hitbox constants for other object
    const otherWidthReduction = otherRect.width * (other.hitbox?.widthPercentage || 0.0);
    const otherHeightReduction = otherRect.height * (other.hitbox?.heightPercentage || 0.0);

    // Build hitbox by subtracting reductions from the left, right, and top
    const thisLeft = thisRect.left + thisWidthReduction;
    const thisTop = thisRect.top + thisHeightReduction;
    const thisRight = thisRect.right - thisWidthReduction;
    const thisBottom = thisRect.bottom;

    const otherLeft = otherRect.left + otherWidthReduction;
    const otherTop = otherRect.top + otherHeightReduction;
    const otherRight = otherRect.right - otherWidthReduction;
    const otherBottom = otherRect.bottom;

    // Determine hit and touch points of hit
    const hit = (
        thisLeft < otherRight &&
        thisRight > otherLeft &&
        thisTop < otherBottom &&
        thisBottom > otherTop
    );
    
    // Rest of the method omitted...
}

/**
 * Handles Player state updates related to the collision
 * Improved to handle mutual collisions between players and prevent clipping
 */
handleCollisionState() {
    // Only process if we have collisions
    if (this.state.collisionEvents.length > 0) {
        const touchPoints = this.collisionData.touchPoints.this;
        const otherObject = this.collisionData.touchPoints.other;
        
        // Get the other object by ID from gameObjects
        const other = this.gameEnv.gameObjects.find(obj => obj.canvas && obj.canvas.id === otherObject.id);
        
        // Calculate overlap amounts to determine push direction
        const bounds = this.canvas.getBoundingClientRect();
        const otherBounds = other?.canvas?.getBoundingClientRect();
        
        if (!otherBounds) return;
        
        // Calculate overlaps in each direction
        const overlapLeft = bounds.right - otherBounds.left;
        const overlapRight = otherBounds.right - bounds.left;
        const overlapTop = bounds.bottom - otherBounds.top;
        const overlapBottom = otherBounds.bottom - bounds.top;
        
        // Find the smallest overlap to determine push direction
        const minOverlap = Math.min(overlapLeft, overlapRight, overlapTop, overlapBottom);
        
        // Handle collision based on the smallest overlap
        if (minOverlap === overlapLeft && touchPoints.right) {
            // We're hitting from the left
            this.state.movement.right = false;
            if (this.velocity && this.velocity.x > 0) {
                this.velocity.x = 0;
            }
            
            // Push objects apart to prevent clipping
            const separation = 1; // Small separation to prevent sticking
            this.position.x = Math.max(0, other.position.x - this.width - separation);
        }
        
        // Similar handling for other directions (omitted for brevity)
    }
}

### Key Points about Collision Handling

1. **Universal Collision System**: The collision system is implemented in the GameObject base class, which both players inherit from

2. **Player-to-Player Collision**: When players collide:
   - Their velocities are set to zero in the direction of collision
   - They're repositioned to prevent clipping into each other
   - The minimum overlap approach ensures natural collision response

3. **Hitbox Customization**: Players can define their hitbox size:
   - The `hitbox` property allows for finer control over collision areas
   - Both players use a small hitbox (`widthPercentage: 0.1, heightPercentage: 0.1`)

4. **Directional Collision**: The system detects which sides are colliding:
   - Top/bottom collisions affect vertical movement
   - Left/right collisions affect horizontal movement

## 5. Game Loop Integration

The game loop in `GameLevel.js` manages the updating of all game objects, including both players:

In [ ]:
%%JS


# GameLevel.js update method

update() {
    this.gameEnv.clear();

    for (let gameObject of this.gameEnv.gameObjects) {
        gameObject.update();
    }

    if (typeof this.gameLevel.update === "function") {
        this.gameLevel.update();
    }
}

### Key Points about Game Loop Integration

1. **Uniform Update Method**: The game loop calls `update()` on every game object
2. **Order Independence**: Objects are updated in the order they were added to the `gameObjects` array
3. **Canvas Clearing**: The canvas is cleared before each frame to prevent visual artifacts
4. **Level-Specific Updates**: The level can define its own update logic if needed

## Summary: Implementing a Two-Player System

Let's summarize the key aspects of implementing a two-player system in the game:

1. **Player Class Separation**:
   - Create separate player classes (PlayerOne and PlayerTwo)
   - Each player handles its own keyboard events
   - Configure different key mappings for each player

2. **Level Configuration**:
   - Define data objects for each player with appropriate properties
   - Set different starting positions on the screen
   - Include both player classes in the level's `classes` array

3. **Visual Differentiation**:
   - Give each player a different appearance (blue vs red squares)
   - Position players on different sides of the screen
   - Ensure each player's canvas has a unique ID

4. **Collision Handling**:
   - Implement a collision detection system that works between different player objects
   - Calculate the minimum overlap to determine the most natural collision response
   - Reposition objects to prevent clipping and ensure fair physics
   - Use velocity adjustment to stop movement in collision directions

5. **Event Handling Separation**:
   - Each player listens to global keyboard events
   - Each player only responds to its specific key codes
   - Key binding happens in each player's constructor
   - Prevents one player's controls from affecting the other

## Practical Implementation Steps

If you want to implement a two-player system in your own game, follow these steps:

1. **Create separate player classes** or use a single class with different configurations:
   ```javascript
   // Option 1: Separate classes
   class PlayerOne extends GameObject { ... }
   class PlayerTwo extends GameObject { ... }
   
   // Option 2: Single class with configuration
   class Player extends GameObject {
     constructor(data) {
       // Use data to configure controls, appearance, etc.
     }
   }
   ```

2. **Configure different controls** for each player:
   ```javascript
   // Player One uses WASD
   const player1Controls = {up: 87, left: 65, down: 83, right: 68};
   
   // Player Two uses IJKL
   const player2Controls = {up: 73, left: 74, down: 75, right: 76};
   ```

3. **Initialize both players** in your game level:
   ```javascript
   this.player1 = new PlayerOne(player1Data, gameEnv);
   this.player2 = new PlayerTwo(player2Data, gameEnv);
   this.gameObjects.push(this.player1, this.player2);
   ```

4. **Implement collision detection** that works between players:
   ```javascript
   // In your update loop
   checkCollisions() {
     // Check if player 1 and player 2 are colliding
     const isColliding = this.checkObjectsCollision(this.player1, this.player2);
     if (isColliding) {
       this.resolveCollision(this.player1, this.player2);
     }
   }
   ```

5. **Ensure fair physics** by handling collisions symmetrically:
   ```javascript
   resolveCollision(obj1, obj2) {
     // Calculate overlap
     // Move both objects back based on the overlap
     // Adjust velocities accordingly
   }
   ```

## Advanced Topics: Extending the Two-Player System

Once you have a basic two-player system working, you might want to extend it with additional features:

1. **Player Interactions**:
   - Implement actions that players can take to affect each other
   - Add combat or cooperative mechanics
   - Create power-ups that can be collected by either player

2. **Scoring System**:
   ```javascript
   class GameLevelSquares {
     constructor(gameEnv) {
       // ... existing code ...
       
       // Add score tracking
       this.scores = { player1: 0, player2: 0 };
       
       // Create score display
       this.createScoreDisplay();
     }
     
     updateScore(player, points) {
       this.scores[player] += points;
       document.getElementById(`${player}-score`).textContent = this.scores[player];
     }
   }
   ```

3. **Game States and Rounds**:
   - Implement round-based gameplay
   - Add win conditions and match tracking
   - Create round transitions and countdowns

4. **Player Customization**:
   - Allow players to choose different characters or abilities
   - Implement character-specific skills or attributes
   - Add visual customization options

## Conclusion

Implementing a two-player system in a JavaScript game involves:

1. Creating separate player classes or configurations
2. Mapping different control schemes for each player
3. Handling player-to-player collisions fairly
4. Updating and rendering each player independently
5. Managing the game state to accommodate multiple players

The GameLevelSquares implementation demonstrates these principles with two simple colored squares controlled by different key sets. This pattern can be extended to more complex games with sprite animations, different player abilities, and more sophisticated physics.

For further learning, consider:
- Adding player scores or health systems
- Implementing player vs. player combat mechanics
- Creating cooperative gameplay elements
- Adding more visual polish to differentiate players

## Hacks

Make sure to do this first hack.
1. Implement the two player system into your own games. one player should use WASD and the other can either use IJKL or arrow keys (up, down, left, right).

### CHOOSE ONE:

2. Add collectible items that can be picked up by either player
3. Create a simple "push" mechanic where players can push each other when colliding, or make it so that collisions stop both players entirely. If you want to make an extension of this, make one player stronger or faster than the other so they can push the other player around!